In [1]:
import os
import orjson



dataset = read_table(os.path.expanduser("~/Data/standard_rated_2013-01_filtered.parquet.gz"))

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject